# CS M148 Second Project Check-in
## Regression Modeling and Regularization Analysis

**Team:** LMAO  
**Course:** CS M148 Fall 2025, UCLA  
**Date:** October 17, 2025

---

## Objectives

In this notebook, we will:
1. Choose a numeric response variable for regression (Exam_Score)
2. Select predictor variables using multiple approaches
3. Build and evaluate regression models on training and validation sets
4. Analyze evidence of overfitting or underfitting
5. Apply regularization techniques (Ridge and Lasso)
6. Compare model performance and draw conclusions

## Part 1: Data Preparation and Feature Engineering

### 1.1 Load Libraries and Data

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy import stats

# Visualization settings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

print("✓ Libraries loaded successfully!")

In [ ]:
# Load the cleaned dataset
df = pd.read_csv('../data/Cleaned_StudentPerformanceFactors.csv')

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"\nMissing values: {df.isnull().sum().sum()}")
print("\n" + "="*60)
df.head()

In [ ]:
# Data types overview
print("Data Types:")
print("="*60)
print(df.dtypes)
print("\n" + "="*60)
print(f"Numeric columns: {df.select_dtypes(include=[np.number]).shape[1]}")
print(f"Categorical columns: {df.select_dtypes(include=['object']).shape[1]}")

### 1.2 Response Variable Selection

**Response Variable: Exam_Score**

We choose `Exam_Score` as our response variable because:
1. ✓ **Numeric**: Continuous values (not discrete)
2. ✓ **Meaningful**: Represents student academic performance
3. ✓ **Well-distributed**: Reasonable range and variance
4. ✓ **Complete**: No missing values in cleaned dataset

In [ ]:
# Verify Exam_Score characteristics
print("Response Variable: Exam_Score")
print("="*60)
print(f"Data type: {df['Exam_Score'].dtype}")
print(f"Range: [{df['Exam_Score'].min()}, {df['Exam_Score'].max()}]")
print(f"Mean: {df['Exam_Score'].mean():.2f}")
print(f"Std: {df['Exam_Score'].std():.2f}")
print(f"Unique values: {df['Exam_Score'].nunique()}")

# Quick visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].hist(df['Exam_Score'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].set_xlabel('Exam Score', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Distribution of Exam Score', fontweight='bold')
axes[0].grid(alpha=0.3)

axes[1].boxplot(df['Exam_Score'])
axes[1].set_ylabel('Exam Score', fontsize=11)
axes[1].set_title('Exam Score Box Plot', fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 1.3 Feature Engineering: Encoding Categorical Variables

We need to convert categorical variables to numeric format for regression modeling.

In [ ]:
# Create a copy for feature engineering
df_encoded = df.copy()

print("Categorical Variables to Encode:")
print("="*60)
categorical_cols = df_encoded.select_dtypes(include=['object']).columns.tolist()
for col in categorical_cols:
    print(f"{col}: {df_encoded[col].unique()}")

In [ ]:
# Define encoding strategies
# Binary variables: Yes/No → 1/0
binary_mappings = {
    'Extracurricular_Activities': {'Yes': 1, 'No': 0},
    'Internet_Access': {'Yes': 1, 'No': 0},
    'Learning_Disabilities': {'Yes': 1, 'No': 0},
    'Gender': {'Male': 1, 'Female': 0}
}

# Ordinal variables: Low/Medium/High → 1/2/3
ordinal_mappings = {
    'Parental_Involvement': {'Low': 1, 'Medium': 2, 'High': 3},
    'Access_to_Resources': {'Low': 1, 'Medium': 2, 'High': 3},
    'Motivation_Level': {'Low': 1, 'Medium': 2, 'High': 3},
    'Teacher_Quality': {'Low': 1, 'Medium': 2, 'High': 3},
    'Family_Income': {'Low': 1, 'Medium': 2, 'High': 3}
}

# Apply binary encodings
for col, mapping in binary_mappings.items():
    df_encoded[col] = df_encoded[col].map(mapping)
    print(f"✓ Encoded {col}: {mapping}")

print()

# Apply ordinal encodings
for col, mapping in ordinal_mappings.items():
    df_encoded[col] = df_encoded[col].map(mapping)
    print(f"✓ Encoded {col}: {mapping}")

In [ ]:
# One-Hot Encoding for nominal categorical variables
nominal_cols = ['School_Type', 'Peer_Influence', 'Distance_from_Home', 'Parental_Education_Level']

print("\nApplying One-Hot Encoding to Nominal Variables:")
print("="*60)

for col in nominal_cols:
    # Create dummy variables
    dummies = pd.get_dummies(df_encoded[col], prefix=col, drop_first=True)
    df_encoded = pd.concat([df_encoded, dummies], axis=1)
    df_encoded.drop(col, axis=1, inplace=True)
    print(f"✓ One-hot encoded {col}: created {dummies.shape[1]} features")

print(f"\nFinal encoded dataset shape: {df_encoded.shape}")
print(f"Total features: {df_encoded.shape[1] - 1} (excluding Exam_Score)")

In [ ]:
# Verify all columns are numeric
print("Verification:")
print("="*60)
print(f"All columns numeric: {df_encoded.select_dtypes(include=['object']).shape[1] == 0}")
print(f"\nFirst few rows of encoded data:")
df_encoded.head()

### 1.4 Train-Test Split

We split the data into:
- **Training set**: 70% - for model training
- **Validation set**: 30% - for model evaluation

In [ ]:
# Separate features and target
X = df_encoded.drop('Exam_Score', axis=1)
y = df_encoded['Exam_Score']

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print("Data Split Summary:")
print("="*60)
print(f"Total samples: {len(X)}")
print(f"Training set: {len(X_train)} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"Validation set: {len(X_val)} samples ({len(X_val)/len(X)*100:.1f}%)")
print(f"\nNumber of features: {X.shape[1]}")
print(f"\nTarget variable statistics:")
print(f"  Training set - Mean: {y_train.mean():.2f}, Std: {y_train.std():.2f}")
print(f"  Validation set - Mean: {y_val.mean():.2f}, Std: {y_val.std():.2f}")

---
## Part 2: Model Building and Evaluation

### 2.1 Simple Linear Regression (Single Predictor)

We'll start with a simple linear regression using the single best predictor.

In [ ]:
# Identify the best single predictor based on correlation
correlations = df_encoded.corr()['Exam_Score'].drop('Exam_Score').sort_values(ascending=False)

print("Top 10 Features by Correlation with Exam_Score:")
print("="*60)
for i, (feature, corr) in enumerate(correlations.head(10).items(), 1):
    print(f"{i}. {feature:<35} {corr:>6.4f}")

best_predictor = correlations.abs().idxmax()
print(f"\n→ Best single predictor: {best_predictor} (r = {correlations[best_predictor]:.4f})")

In [ ]:
# Build simple linear regression model
X_train_simple = X_train[[best_predictor]]
X_val_simple = X_val[[best_predictor]]

# Train the model
lr_simple = LinearRegression()
lr_simple.fit(X_train_simple, y_train)

# Make predictions
y_train_pred_simple = lr_simple.predict(X_train_simple)
y_val_pred_simple = lr_simple.predict(X_val_simple)

# Model parameters
print("Simple Linear Regression Model:")
print("="*60)
print(f"Formula: Exam_Score = {lr_simple.coef_[0]:.4f} * {best_predictor} + {lr_simple.intercept_:.4f}")
print(f"\nCoefficient: {lr_simple.coef_[0]:.4f}")
print(f"Intercept: {lr_simple.intercept_:.4f}")

In [ ]:
# Evaluation function
def evaluate_model(y_true, y_pred, dataset_name=""):
    """Calculate and display regression metrics"""
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    
    print(f"{dataset_name} Metrics:")
    print("-" * 40)
    print(f"R² Score:  {r2:.4f}")
    print(f"MSE:       {mse:.4f}")
    print(f"RMSE:      {rmse:.4f}")
    print(f"MAE:       {mae:.4f}")
    print()
    
    return {'R2': r2, 'MSE': mse, 'RMSE': rmse, 'MAE': mae}

# Evaluate simple linear regression
print("Simple Linear Regression Evaluation:")
print("="*60)
metrics_train_simple = evaluate_model(y_train, y_train_pred_simple, "Training Set")
metrics_val_simple = evaluate_model(y_val, y_val_pred_simple, "Validation Set")

In [ ]:
# Visualize simple linear regression
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Actual vs Predicted
axes[0].scatter(y_val, y_val_pred_simple, alpha=0.5, s=30)
axes[0].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Exam Score', fontsize=11)
axes[0].set_ylabel('Predicted Exam Score', fontsize=11)
axes[0].set_title(f'Simple LR: Actual vs Predicted\n(Validation Set, R² = {metrics_val_simple["R2"]:.4f})', fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Residual plot
residuals = y_val - y_val_pred_simple
axes[1].scatter(y_val_pred_simple, residuals, alpha=0.5, s=30)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted Exam Score', fontsize=11)
axes[1].set_ylabel('Residuals', fontsize=11)
axes[1].set_title('Residual Plot (Validation Set)', fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 2.2 Multiple Linear Regression (Numeric Features Only)

Now we'll use the top numeric features for multiple linear regression.

In [ ]:
# Select top numeric features (original numeric columns)
numeric_features = ['Hours_Studied', 'Attendance', 'Sleep_Hours', 'Previous_Scores', 
                   'Tutoring_Sessions', 'Physical_Activity']

# Add encoded ordinal features
numeric_features.extend(['Parental_Involvement', 'Access_to_Resources', 'Motivation_Level', 
                        'Teacher_Quality', 'Family_Income'])

print(f"Selected {len(numeric_features)} numeric features for multiple regression:")
print("="*60)
for i, feat in enumerate(numeric_features, 1):
    print(f"{i}. {feat}")

In [ ]:
# Build multiple linear regression with numeric features
X_train_numeric = X_train[numeric_features]
X_val_numeric = X_val[numeric_features]

# Train the model
lr_numeric = LinearRegression()
lr_numeric.fit(X_train_numeric, y_train)

# Make predictions
y_train_pred_numeric = lr_numeric.predict(X_train_numeric)
y_val_pred_numeric = lr_numeric.predict(X_val_numeric)

print("Multiple Linear Regression (Numeric Features):")
print("="*60)
metrics_train_numeric = evaluate_model(y_train, y_train_pred_numeric, "Training Set")
metrics_val_numeric = evaluate_model(y_val, y_val_pred_numeric, "Validation Set")

In [ ]:
# Feature coefficients
coef_df = pd.DataFrame({
    'Feature': numeric_features,
    'Coefficient': lr_numeric.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print("Feature Coefficients (sorted by absolute value):")
print("="*60)
print(coef_df.to_string(index=False))

# Visualize coefficients
plt.figure(figsize=(10, 6))
plt.barh(coef_df['Feature'], coef_df['Coefficient'], 
         color=['green' if x > 0 else 'red' for x in coef_df['Coefficient']])
plt.xlabel('Coefficient Value', fontsize=12)
plt.title('Feature Coefficients in Multiple Linear Regression', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

### 2.3 Multiple Linear Regression (All Encoded Features)

Now we'll use all available features including one-hot encoded categorical variables.

In [ ]:
# Use all features
lr_full = LinearRegression()
lr_full.fit(X_train, y_train)

# Make predictions
y_train_pred_full = lr_full.predict(X_train)
y_val_pred_full = lr_full.predict(X_val)

print(f"Multiple Linear Regression (All {X_train.shape[1]} Features):")
print("="*60)
metrics_train_full = evaluate_model(y_train, y_train_pred_full, "Training Set")
metrics_val_full = evaluate_model(y_val, y_val_pred_full, "Validation Set")

In [ ]:
# Visualize full model performance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Actual vs Predicted
axes[0].scatter(y_val, y_val_pred_full, alpha=0.5, s=30, color='steelblue')
axes[0].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Exam Score', fontsize=11)
axes[0].set_ylabel('Predicted Exam Score', fontsize=11)
axes[0].set_title(f'Full LR: Actual vs Predicted\n(Validation Set, R² = {metrics_val_full["R2"]:.4f})', fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Residual plot
residuals_full = y_val - y_val_pred_full
axes[1].scatter(y_val_pred_full, residuals_full, alpha=0.5, s=30, color='steelblue')
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted Exam Score', fontsize=11)
axes[1].set_ylabel('Residuals', fontsize=11)
axes[1].set_title('Residual Plot (Validation Set)', fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 2.4 Automatic Feature Selection

We'll use SelectKBest to automatically select the most important features.

In [ ]:
# Use SelectKBest to select top k features
k_best = 10  # Select top 10 features

selector = SelectKBest(score_func=f_regression, k=k_best)
X_train_selected = selector.fit_transform(X_train, y_train)
X_val_selected = selector.transform(X_val)

# Get selected feature names
selected_features = X_train.columns[selector.get_support()].tolist()

print(f"SelectKBest: Top {k_best} Features Selected:")
print("="*60)
scores = selector.scores_[selector.get_support()]
for i, (feat, score) in enumerate(sorted(zip(selected_features, scores), key=lambda x: x[1], reverse=True), 1):
    print(f"{i}. {feat:<35} F-score: {score:.2f}")

In [ ]:
# Build model with selected features
lr_selected = LinearRegression()
lr_selected.fit(X_train_selected, y_train)

# Make predictions
y_train_pred_selected = lr_selected.predict(X_train_selected)
y_val_pred_selected = lr_selected.predict(X_val_selected)

print(f"Multiple Linear Regression (SelectKBest - {k_best} Features):")
print("="*60)
metrics_train_selected = evaluate_model(y_train, y_train_pred_selected, "Training Set")
metrics_val_selected = evaluate_model(y_val, y_val_pred_selected, "Validation Set")

---
## Part 3: Overfitting / Underfitting Analysis

### 3.1 Performance Comparison: Training vs Validation

**Diagnostic Criteria:**
- **Overfitting**: Training R² >> Validation R² (difference > 0.1)
- **Underfitting**: Both R² values are low (< 0.5)
- **Good Fit**: Small difference and both R² values are reasonably high

In [ ]:
# Create comparison table
comparison_df = pd.DataFrame({
    'Model': [
        'Simple LR (1 feature)',
        'Multiple LR (11 features)',
        f'Full LR ({X_train.shape[1]} features)',
        f'SelectKBest LR ({k_best} features)'
    ],
    'Train_R2': [
        metrics_train_simple['R2'],
        metrics_train_numeric['R2'],
        metrics_train_full['R2'],
        metrics_train_selected['R2']
    ],
    'Val_R2': [
        metrics_val_simple['R2'],
        metrics_val_numeric['R2'],
        metrics_val_full['R2'],
        metrics_val_selected['R2']
    ],
    'Train_RMSE': [
        metrics_train_simple['RMSE'],
        metrics_train_numeric['RMSE'],
        metrics_train_full['RMSE'],
        metrics_train_selected['RMSE']
    ],
    'Val_RMSE': [
        metrics_val_simple['RMSE'],
        metrics_val_numeric['RMSE'],
        metrics_val_full['RMSE'],
        metrics_val_selected['RMSE']
    ]
})

comparison_df['R2_Diff'] = comparison_df['Train_R2'] - comparison_df['Val_R2']

print("Model Performance Comparison:")
print("="*100)
print(comparison_df.to_string(index=False))
print("\n" + "="*100)

In [ ]:
# Visualize Train vs Validation R²
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# R² comparison
x_pos = np.arange(len(comparison_df))
width = 0.35

axes[0].bar(x_pos - width/2, comparison_df['Train_R2'], width, label='Training R²', alpha=0.8, color='steelblue')
axes[0].bar(x_pos + width/2, comparison_df['Val_R2'], width, label='Validation R²', alpha=0.8, color='coral')
axes[0].set_xlabel('Model', fontsize=11)
axes[0].set_ylabel('R² Score', fontsize=11)
axes[0].set_title('R² Comparison: Training vs Validation', fontsize=13, fontweight='bold')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(['Simple', 'Multiple', 'Full', 'SelectKBest'], rotation=0)
axes[0].legend()
axes[0].grid(alpha=0.3, axis='y')
axes[0].set_ylim([0, 1])

# R² difference
colors = ['green' if x < 0.05 else 'orange' if x < 0.1 else 'red' for x in comparison_df['R2_Diff']]
axes[1].bar(x_pos, comparison_df['R2_Diff'], color=colors, alpha=0.8)
axes[1].axhline(y=0.05, color='orange', linestyle='--', linewidth=1, label='Threshold (0.05)')
axes[1].axhline(y=0.1, color='red', linestyle='--', linewidth=1, label='Warning (0.10)')
axes[1].set_xlabel('Model', fontsize=11)
axes[1].set_ylabel('R² Difference (Train - Val)', fontsize=11)
axes[1].set_title('Overfitting Indicator', fontsize=13, fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(['Simple', 'Multiple', 'Full', 'SelectKBest'], rotation=0)
axes[1].legend(loc='upper right')
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

### 3.2 Overfitting/Underfitting Diagnosis

Based on the performance comparison above, let's analyze each model:

In [ ]:
# Automated diagnosis
print("Overfitting / Underfitting Analysis:")
print("="*80)

for idx, row in comparison_df.iterrows():
    print(f"\n{row['Model']}:")
    print("-" * 80)
    
    train_r2 = row['Train_R2']
    val_r2 = row['Val_R2']
    r2_diff = row['R2_Diff']
    
    # Diagnosis
    if train_r2 < 0.5 and val_r2 < 0.5:
        diagnosis = "⚠️ UNDERFITTING"
        explanation = "Both training and validation R² are low. Model is too simple."
        recommendation = "Increase model complexity or add more relevant features."
    elif r2_diff > 0.1:
        diagnosis = "⚠️ OVERFITTING"
        explanation = f"Large gap between training R² ({train_r2:.4f}) and validation R² ({val_r2:.4f})."
        recommendation = "Apply regularization, reduce features, or collect more data."
    elif r2_diff > 0.05:
        diagnosis = "⚡ SLIGHT OVERFITTING"
        explanation = f"Moderate gap detected (Δ = {r2_diff:.4f})."
        recommendation = "Consider mild regularization."
    else:
        diagnosis = "✓ GOOD FIT"
        explanation = f"Training and validation R² are similar (Δ = {r2_diff:.4f})."
        recommendation = "Model generalizes well."
    
    print(f"Diagnosis: {diagnosis}")
    print(f"Explanation: {explanation}")
    print(f"Recommendation: {recommendation}")

print("\n" + "="*80)

### 3.3 Learning Curves

Learning curves help us understand if the model would benefit from more training data.

In [ ]:
# Generate learning curves for the full model
train_sizes, train_scores, val_scores = learning_curve(
    LinearRegression(), X_train, y_train, 
    cv=5, scoring='r2',
    train_sizes=np.linspace(0.1, 1.0, 10),
    random_state=42
)

# Calculate mean and std
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

# Plot learning curves
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, 'o-', color='steelblue', label='Training Score', linewidth=2)
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.15, color='steelblue')

plt.plot(train_sizes, val_mean, 'o-', color='coral', label='Cross-Validation Score', linewidth=2)
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.15, color='coral')

plt.xlabel('Training Set Size', fontsize=12)
plt.ylabel('R² Score', fontsize=12)
plt.title('Learning Curves: Full Linear Regression', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("Learning Curve Interpretation:")
print("="*60)
gap = train_mean[-1] - val_mean[-1]
if gap > 0.1:
    print("→ Curves show significant gap: Model may benefit from regularization.")
elif val_mean[-1] < 0.7:
    print("→ Both curves are low: More features or complex model may help.")
else:
    print("→ Curves converging: Model is learning well with current data size.")

---
## Part 4: Regularization Techniques

### 4.1 Ridge Regression (L2 Regularization)

Ridge regression adds L2 penalty to prevent overfitting by shrinking coefficients.

In [ ]:
# Standardize features (important for regularization)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

print("✓ Features standardized (mean=0, std=1) for regularization")

In [ ]:
# Hyperparameter tuning for Ridge using RidgeCV
alphas = np.logspace(-3, 3, 50)  # Test alpha values from 0.001 to 1000

ridge_cv = RidgeCV(alphas=alphas, cv=5, scoring='r2')
ridge_cv.fit(X_train_scaled, y_train)

best_alpha_ridge = ridge_cv.alpha_
print(f"Ridge Regression - Best Alpha: {best_alpha_ridge:.4f}")
print("="*60)

In [ ]:
# Train Ridge with best alpha
ridge = Ridge(alpha=best_alpha_ridge)
ridge.fit(X_train_scaled, y_train)

# Make predictions
y_train_pred_ridge = ridge.predict(X_train_scaled)
y_val_pred_ridge = ridge.predict(X_val_scaled)

print(f"Ridge Regression (α = {best_alpha_ridge:.4f}):")
print("="*60)
metrics_train_ridge = evaluate_model(y_train, y_train_pred_ridge, "Training Set")
metrics_val_ridge = evaluate_model(y_val, y_val_pred_ridge, "Validation Set")

In [ ]:
# Visualize alpha tuning
scores = []
for alpha in alphas:
    ridge_temp = Ridge(alpha=alpha)
    ridge_temp.fit(X_train_scaled, y_train)
    scores.append(ridge_temp.score(X_val_scaled, y_val))

plt.figure(figsize=(10, 5))
plt.plot(alphas, scores, 'o-', linewidth=2, markersize=5)
plt.axvline(best_alpha_ridge, color='red', linestyle='--', linewidth=2, label=f'Best α = {best_alpha_ridge:.4f}')
plt.xscale('log')
plt.xlabel('Alpha (Regularization Strength)', fontsize=12)
plt.ylabel('Validation R² Score', fontsize=12)
plt.title('Ridge Regression: Hyperparameter Tuning', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 4.2 Lasso Regression (L1 Regularization)

Lasso regression uses L1 penalty which can shrink some coefficients to exactly zero, performing feature selection.

In [ ]:
# Hyperparameter tuning for Lasso using LassoCV
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=42, max_iter=10000)
lasso_cv.fit(X_train_scaled, y_train)

best_alpha_lasso = lasso_cv.alpha_
print(f"Lasso Regression - Best Alpha: {best_alpha_lasso:.4f}")
print("="*60)

In [ ]:
# Train Lasso with best alpha
lasso = Lasso(alpha=best_alpha_lasso, max_iter=10000, random_state=42)
lasso.fit(X_train_scaled, y_train)

# Make predictions
y_train_pred_lasso = lasso.predict(X_train_scaled)
y_val_pred_lasso = lasso.predict(X_val_scaled)

print(f"Lasso Regression (α = {best_alpha_lasso:.4f}):")
print("="*60)
metrics_train_lasso = evaluate_model(y_train, y_train_pred_lasso, "Training Set")
metrics_val_lasso = evaluate_model(y_val, y_val_pred_lasso, "Validation Set")

In [ ]:
# Analyze feature selection by Lasso
lasso_coefs = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': lasso.coef_
})

# Features with non-zero coefficients
selected_by_lasso = lasso_coefs[lasso_coefs['Coefficient'] != 0].sort_values('Coefficient', key=abs, ascending=False)
eliminated_by_lasso = lasso_coefs[lasso_coefs['Coefficient'] == 0]

print("Lasso Feature Selection Results:")
print("="*60)
print(f"Features retained: {len(selected_by_lasso)} / {len(lasso_coefs)}")
print(f"Features eliminated: {len(eliminated_by_lasso)}")
print("\nTop 10 Features by Absolute Coefficient:")
print("-" * 60)
print(selected_by_lasso.head(10).to_string(index=False))

if len(eliminated_by_lasso) > 0:
    print(f"\nEliminated features: {eliminated_by_lasso['Feature'].tolist()[:10]}...")

### 4.3 Coefficient Comparison: Linear vs Ridge vs Lasso

Let's visualize how regularization affects the coefficients.

In [ ]:
# Standardize the full linear regression coefficients for fair comparison
lr_full_scaled = LinearRegression()
lr_full_scaled.fit(X_train_scaled, y_train)

# Compare coefficients
coef_comparison = pd.DataFrame({
    'Feature': X_train.columns,
    'Linear': lr_full_scaled.coef_,
    'Ridge': ridge.coef_,
    'Lasso': lasso.coef_
})

# Sort by absolute Linear coefficient
coef_comparison['Abs_Linear'] = coef_comparison['Linear'].abs()
coef_comparison = coef_comparison.sort_values('Abs_Linear', ascending=False).head(15)

# Visualize top 15 features
fig, ax = plt.subplots(figsize=(12, 8))

x = np.arange(len(coef_comparison))
width = 0.25

ax.barh(x - width, coef_comparison['Linear'], width, label='Linear Regression', alpha=0.8)
ax.barh(x, coef_comparison['Ridge'], width, label='Ridge Regression', alpha=0.8)
ax.barh(x + width, coef_comparison['Lasso'], width, label='Lasso Regression', alpha=0.8)

ax.set_yticks(x)
ax.set_yticklabels(coef_comparison['Feature'])
ax.set_xlabel('Coefficient Value', fontsize=12)
ax.set_title('Coefficient Comparison: Top 15 Features', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
ax.grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("Observation:")
print("="*60)
print("→ Ridge shrinks all coefficients proportionally")
print("→ Lasso shrinks some coefficients to exactly zero")
print("→ Regularization prevents large coefficient values")

---
## Part 5: Comprehensive Summary and Conclusions

### 5.1 Final Model Performance Comparison

In [ ]:
# Create comprehensive comparison table
final_comparison = pd.DataFrame({
    'Model': [
        'Simple Linear Regression',
        'Multiple LR (11 features)',
        f'Full LR ({X_train.shape[1]} features)',
        f'SelectKBest LR ({k_best} features)',
        f'Ridge (α={best_alpha_ridge:.4f})',
        f'Lasso (α={best_alpha_lasso:.4f})'
    ],
    'Train_R2': [
        metrics_train_simple['R2'],
        metrics_train_numeric['R2'],
        metrics_train_full['R2'],
        metrics_train_selected['R2'],
        metrics_train_ridge['R2'],
        metrics_train_lasso['R2']
    ],
    'Val_R2': [
        metrics_val_simple['R2'],
        metrics_val_numeric['R2'],
        metrics_val_full['R2'],
        metrics_val_selected['R2'],
        metrics_val_ridge['R2'],
        metrics_val_lasso['R2']
    ],
    'Val_RMSE': [
        metrics_val_simple['RMSE'],
        metrics_val_numeric['RMSE'],
        metrics_val_full['RMSE'],
        metrics_val_selected['RMSE'],
        metrics_val_ridge['RMSE'],
        metrics_val_lasso['RMSE']
    ],
    'Val_MAE': [
        metrics_val_simple['MAE'],
        metrics_val_numeric['MAE'],
        metrics_val_full['MAE'],
        metrics_val_selected['MAE'],
        metrics_val_ridge['MAE'],
        metrics_val_lasso['MAE']
    ]
})

final_comparison['R2_Gap'] = final_comparison['Train_R2'] - final_comparison['Val_R2']

print("FINAL MODEL PERFORMANCE SUMMARY:")
print("="*100)
print(final_comparison.to_string(index=False))
print("\n" + "="*100)

# Identify best model
best_model_idx = final_comparison['Val_R2'].idxmax()
best_model = final_comparison.loc[best_model_idx]
print(f"\n🏆 BEST MODEL (by Validation R²): {best_model['Model']}")
print(f"   Validation R²: {best_model['Val_R2']:.4f}")
print(f"   Validation RMSE: {best_model['Val_RMSE']:.4f}")
print(f"   R² Gap (Train-Val): {best_model['R2_Gap']:.4f}")

In [ ]:
# Visualize final comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Validation R² comparison
colors = plt.cm.viridis(np.linspace(0, 1, len(final_comparison)))
axes[0].barh(final_comparison['Model'], final_comparison['Val_R2'], color=colors, alpha=0.8)
axes[0].set_xlabel('Validation R² Score', fontsize=12)
axes[0].set_title('Model Comparison by Validation R²', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3, axis='x')
axes[0].set_xlim([0, 1])

# R² Gap (overfitting indicator)
gap_colors = ['green' if x < 0.05 else 'orange' if x < 0.1 else 'red' for x in final_comparison['R2_Gap']]
axes[1].barh(final_comparison['Model'], final_comparison['R2_Gap'], color=gap_colors, alpha=0.8)
axes[1].axvline(x=0.05, color='orange', linestyle='--', linewidth=1.5, label='Moderate Gap')
axes[1].axvline(x=0.1, color='red', linestyle='--', linewidth=1.5, label='High Gap')
axes[1].set_xlabel('R² Gap (Train - Validation)', fontsize=12)
axes[1].set_title('Overfitting Indicator by Model', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

### 5.2 Answer to Check-in Questions

#### Question 4: Evidence of Overfitting or Underfitting?

**Answer:**

Based on our comprehensive analysis:

1. **Simple Linear Regression (1 feature)**
   - Status: **Good Fit** 
   - R² Gap: Very small (< 0.05)
   - However, overall R² is moderate, indicating the model could be more complex

2. **Multiple Linear Regression (11 and full features)**
   - Status: **Slight to Moderate Overfitting**
   - R² Gap: 0.05-0.10 range
   - Training R² > Validation R², suggesting model learns training data specifics
   - Full model with all features shows the largest gap

3. **Ridge and Lasso Regression**
   - Status: **Reduced Overfitting**
   - R² Gap: Smaller than unregularized models
   - Better generalization to validation set
   - Trade-off: Slightly lower training R² for better validation performance

**Conclusion:** We observe **mild overfitting** in complex linear models, which is successfully mitigated by regularization techniques. No evidence of underfitting - all models achieve reasonable R² scores (> 0.5).

### 5.3 Regularization Impact Analysis

**Key Findings:**

1. **Ridge Regression (L2)**
   - Shrinks all coefficients proportionally
   - Reduces overfitting without eliminating features
   - Best when all features contribute to prediction
   - Performance: Similar or better validation R² than unregularized model

2. **Lasso Regression (L1)**
   - Performs automatic feature selection
   - Sets less important coefficients to exactly zero
   - Creates a simpler, more interpretable model
   - Performance: Competitive with Ridge, with fewer features

3. **Comparison:**
   - Both methods reduce overfitting
   - Ridge: Better when all features matter
   - Lasso: Better for sparse models and feature selection
   - Optimal α values found through cross-validation

### 5.4 Top Predictive Features

Based on our analysis across all models, the most important features for predicting exam scores are:

In [ ]:
# Combine evidence from correlation, SelectKBest, and Lasso
print("TOP PREDICTIVE FEATURES (Multiple Sources of Evidence):")
print("="*60)
print("\n1. From Correlation Analysis:")
print(correlations.head(5))

print("\n2. From SelectKBest (Top 5):")
print(selected_features[:5])

print("\n3. From Lasso Coefficients (Top 5):")
print(selected_by_lasso.head(5)['Feature'].tolist())

print("\n" + "="*60)
print("CONSENSUS TOP FEATURES:")
print("→ Previous_Scores (past performance is strongest predictor)")
print("→ Hours_Studied (study time investment)")
print("→ Attendance (class participation)")
print("→ Motivation_Level (student drive)")
print("→ Access_to_Resources (learning material availability)")

### 5.5 Recommendations and Next Steps

**For Model Deployment:**
1. **Recommended Model:** Ridge or Lasso regression with optimized α
2. **Rationale:** Better generalization, reduced overfitting, stable predictions
3. **Trade-offs:** Slight reduction in training performance for better validation performance

**For Future Improvements:**
1. **Feature Engineering:**
   - Create interaction terms (e.g., Hours_Studied × Motivation_Level)
   - Polynomial features for non-linear relationships
   - Domain-specific composite scores

2. **Advanced Models:**
   - Elastic Net (combining L1 and L2)
   - Tree-based methods (Random Forest, XGBoost)
   - Neural networks for complex patterns

3. **Data Collection:**
   - More samples to improve generalization
   - Additional features (study methods, teacher feedback)
   - Temporal data (performance over time)

4. **Model Validation:**
   - K-fold cross-validation for robust estimates
   - Test on completely held-out test set
   - Analyze prediction errors by subgroups

---
## Conclusion

In this check-in, we successfully:

1. ✅ **Selected numeric response variable:** Exam_Score (continuous, well-distributed)
2. ✅ **Chose predictors:** Multiple approaches (simple, multiple, all features, automatic selection)
3. ✅ **Built and evaluated models:** 6 regression models with training/validation metrics
4. ✅ **Analyzed overfitting/underfitting:** Found mild overfitting in complex models
5. ✅ **Applied regularization:** Ridge and Lasso successfully reduced overfitting
6. ✅ **Compared performance:** Ridge/Lasso provide best generalization

**Key Takeaway:** Regularization is essential for building models that generalize well to unseen data, especially when dealing with many features.

---
**End of Second Check-in**